<a href="https://colab.research.google.com/github/mahekp05/Reinforcement-Learning-Projects/blob/main/SUCESS_taxi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## 1. Install Packages and Dependencies

*   **gymnaisum**: toolkit for developing and comparing reinforcement learning algorithms. It provides various environments that simulate real-world scenarios, which are essential for testing and training RL models.
*   **pygame**: writing video games; in reinforcement learning it is used to create custom environments
*   **numpy**: library for numerical computation in python and provides support for arrays, matrices, and many mathematical functions essential for implementing and testing algorithms
*   **imageio and imageio_ffmpeg**: libararies used to read and write images and videos -- useful to render environment and create visualization of our reinforcement learning model and training episodes on collabnotes as live redering is not possible
*   **pyglet**: A cross-platform windowing and multimedia library -- used for rendering graphics in reinforcement learning environments
*   **tqdm**: creates progress bars to help monitor progress of training and lenghty episodes

In [ ]:
pip install gymnasium pygame numpy pickle5 imageio imageio_ffmpeg pyglet==1.5.1 tqdm

In [ ]:
!sudo apt-get update
!sude apt-get install -y python3-opengl
!apt install ffmpeg xvfb
!pip3 install pyvirtualdisplay

In [ ]:
import os
os.kill(os.getpid(),9)

Import libraries

In [ ]:
import numpy as np
import gymnasium as gym
import random
import imageio
import os
import tqdm

import pickle5 as pickle
from tqdm.notebook import tqdm

Virtual display

In [ ]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(3840, 2610))
display.start()

Environment -- gymnaisum

initialize env

In [ ]:
env = gym.make("Taxi-v3", render_mode = 'rgb_array')

Grid Size : 5x5

Passenger locations:

0: Red

1: Green

2: Yellow

3: Blue

4: In taxi

Destinations:

0: Red

1: Green

2: Yellow

3: Blue



In [ ]:
state_space = env.observation_space.n
print("There are ", state_space, "possible states")
#5*4*4*5 = 500

There are  500 possible states


In [ ]:
action_space = env.action_space.n
print("There are", action_space, "possible actions")

There are 6 possible actions


initialize q-table
**bold text**
*q*-table = state-action pair value
Every time agent takes an action, the q-table is  getting optimal q-function/table

In [ ]:
def initialize_q_table(state_space, action_space):
  Qtable = np.zeros((state_space, action_space))
  return Qtable

In [ ]:
Qtable_taxi = initialize_q_table(state_space, action_space)
print(Qtable_taxi)
print("Q-table shape:", Qtable_taxi.shape)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
Q-table shape: (500, 6)


Training and Execution

Training Policies

Off-policy algorithms: diff policy used at training time and inference time

Greedy Policy -- action that will give us highest reward based on current knowledge (only exploitation)

Epsilon Greedy Policy --
exploration vs exploitation
exploration -- learn new things

0 < episolon < 1 (very exploriative)

Greedy policy

In [ ]:
def greedy_policy(Qtable, state):
  #Exploitation: take action with highest state, action value
  action = np.argmax(Qtable[state][:])

  return action

epsilon greedy policy

In [ ]:
def epsilon_greedy_policy(Qtable, state, epsilon):
  #Randomly generate a number between 0 and 1
  random_num = random.uniform(0,1)

  if random_num > epsilon: #exploitation
    #take action with highest value given state
    action = greedy_policy(Qtable, state)
  else: #exploration
    action = env.action_space.sample()

  return action

hyper parameters

In [ ]:
#Training parameters
n_training_episodes = 1500 #total learning episodes
learning_rate = 0.7 #learning rate

#Evaluation parameters
n_eval_episodes = 100 #total number of test episodes

eval_seed = [16,54,165,177,191,191,120,80,149,178,48,38,6,125,174,73,50,172,100,
             148,146,6,25,40,68,148,49,167,9,97,164,176,61,7,54,55,161,131,184,
             51,170,12,120,113,95,126,51,98,36,135,54,82,45,95,89,59,95,124,9,
             113,58,85,51,134,121,169,105,21,30,11,50,65,12,43,82,145,152,97,106,
             55,31,85,38,112,102,168,123,97,21,83,158,26,80,63,5,81,32,11,28,148]
# Evaluation seed, this ensures that all classmates agents are trained on the same taxi starting position
# Each seed has a specific starting state

#Environment parameters
env_id = "Taxi-v3" # name of environment
max_steps = 99 #max steps per episode
gamma = 0.95 #discounting rate

#Exploration parameters
max_epsilon = 1.0 #exploration probability at start
min_epsilon = 0.01 # minimum exploration probability
decay_rate = 0.005 #exploration decay rate for exploration probability

Training function

In [ ]:
def train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env,
          max_steps, Qtable):
  for episode in tqdm(range(n_training_episodes)):
    #Reduce epsilon (bc we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)

    #Reset environment
    state, info = env.reset()
    step = 0
    terminated = False
    truncated = False

    #repeat -- update after each step instead of episode
    for step in range(max_steps):
      #choose the action At using epsilon greedy policy
      action = epsilon_greedy_policy(Qtable, state, epsilon)

      #Take action AT and observe Rt+1 and St+1
      #Take the action (a) and observe the outcome states (s') and reward(r)
      new_state, reward,terminated, truncated, info = env.step(action)

      #Update Q(s,a):= Q(s,a) + lr[R(s,a) + gamma * max Q(s',a') - Q(s,a)]
      Qtable[state][action] = Qtable[state][action] + learning_rate * (reward + gamma * np.max(Qtable[new_state] )- Qtable[state][action])

      #if terminated or truncated finish episode
      if terminated or truncated:
        break

      state = new_state

  return Qtable


Train

In [ ]:
Qtable_taxi = train(n_training_episodes, min_epsilon, max_epsilon, decay_rate,
                    env, max_steps, Qtable_taxi)

Qtable_taxi

  0%|          | 0/1500 [00:00<?, ?it/s]

array([[  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  2.44182855,  -4.26213084,   0.598163  ,   1.33253053,
          5.20997639,  -7.60266139],
       [ -3.21090951,   2.63612139,   4.20230678,  -3.12047287,
         10.9512375 ,  -2.07887104],
       ...,
       [ -2.86848537,  11.70839455,  -2.65639999,  -2.52163774,
        -11.24714343, -10.0147075 ],
       [ -3.71225095,  -3.69301622,  -4.07766031,  -3.55440688,
         -7.        , -10.61725198],
       [ 15.60262615,   9.46034914,  15.94471648,  18.        ,
         -7.        ,   5.2977261 ]])

Model card

In [ ]:
model = {
    "env_id": env_id,
    "max_steps": max_steps,
    "n_training_episodes": n_training_episodes,
    "n_eval_episodes": n_eval_episodes,
    "eval_seed": eval_seed,

    "learning_rate": learning_rate,
    "gamma": gamma,

    "max_epsilon": max_epsilon,
    "min_epsilon": min_epsilon,
    "decay_rate": decay_rate,

    "qtable": Qtable_taxi
}

Record Video

In [ ]:
from math import trunc
def record_video(env, Qtable, out_directory, fps=1, max_steps=20):
  """
  Generate a replay video of agent
  :param env
  :param Qtable: Qtable of our agent
  :param out_directory
  :param fps: how many frames per second (w taxi-v3 we use 1)
  :param max_steps: max steps for recording
  """

  images = []
  terminated = False
  trucanted = False
  state, info = env.reset(seed=random.randint(0,500))
  img = env.render()
  images.append(img)
  step_count = 0

  while(not terminated or not trucanted) and step_count < max_steps:
    action = np.argmax(Qtable[state])
    state, reward,terminated,trucanted,info = env.step(action)
    img = env.render()
    images.append(img)
    step_count +=1

    imageio.mimsave(out_directory, [np.array(img) for i, img in enumerate(images)], fps=fps)


Record single video

In [ ]:
 video_name = "replay.mp4"
 record_video(env, model["qtable"], video_name,1)

In [ ]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('replay.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

Agent Eval -- quantification

In [ ]:
def evaluate_agent(env, max_steps, n_eval_episodes, Q, seed):
  """
  Evaluate the agent for ``n_eval_episodes`` episodes and returns average reward and std of reward.
  :param env: The evaluation environment
  :param max_steps: Maximum number of steps per episode
  :param n_eval_episodes: Number of episode to evaluate the agent
  :param Q: The Q-table
  :param seed: The evaluation seed array (for taxi-v3)
  """
  episode_rewards = []
  for episode in tqdm(range(n_eval_episodes)):
    if seed:
      state, info = env.reset(seed=seed[episode])
    else:
      state, info = env.reset()

    step = 0
    trucanted = False
    terminated = False
    total_rewards_ep = 0

    for step in range(max_steps):
      action = greedy_policy(Q,state)
      new_state, reward, terminated, trucanted, info = env.step(action)
      total_rewards_ep += reward

      if terminated or trucanted:
        break

      state = new_state
      episode_rewards.append(total_rewards_ep)

  mean_reward = np.mean(episode_rewards)
  std_reward = np.std(episode_rewards)

  return mean_reward, std_reward


def train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, max_steps, Qtable):
    # Loop over each training episode
    for episode in tqdm(range(n_training_episodes)):
        # Calculate the current epsilon for exploration-exploitation balance
        epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay_rate * episode)
        
        # Reset the environment to start a new episode
        state, info = env.reset()
        step = 0
        terminated = truncated = False
        
        # Loop over each step in the episode
        for step in range(max_steps):
            # Choose an action using epsilon-greedy policy
            action = epsilon_greedy_policy(Qtable, state, epsilon)
            
            # Take the action and observe the new state and reward
            new_state, reward, terminated, truncated, info = env.step(action)
            
            # Update the Q-table using the Q-learning formula
            Qtable[state][action] = Qtable[state][action] + learning_rate * (
                reward + gamma * np.max(Qtable[new_state]) - Qtable[state][action])
            
            # If the episode is terminated or truncated, end the episode
            if terminated or truncated:
                break
            
            # Move to the new state
            state = new_state
    
    # Return the updated Q-table after all episodes
    return Qtable


In [ ]:
#eval agent
mean_reward, std_reward = evaluate_agent(env, max_steps,n_eval_episodes,
                                         Qtable_taxi, eval_seed)
print(f"Mean_reward={mean_reward:.2f} +/- {std_reward:.2f}")

  0%|          | 0/100 [00:00<?, ?it/s]

Mean_reward=-7.01 +/- 4.02
